In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load the dataset
folder_path = "/content/WSN-DS.csv"
dataset = pd.read_csv(folder_path)

# Prepare features and target variable
X = dataset.drop(['Attack type'], axis=1)
Y = dataset['Attack type']


In [ ]:
# remove spaces from the beginning of the column names
data.columns = data.columns.str.lstrip()
# rename the Attack type column to label
data.rename(columns={'Attack type': 'label'}, inplace=True)
 data.drop(['id', 'send_code '], axis=1, inplace=True)
  data.isnull().sum() # check for missing values it gave 0 for all columns
data.drop_duplicates(inplace=True) # remove duplicates it removed 8,873 rows


In [ ]:
X = data.drop('label', axis=1)
y = data['label']
class_counts = {'Normal': 332040, 'Grayhole': 13909, 'Blackhole': 10049, 'TDMA':
6633, 'Flooding': 3157}
# Calculate the number of samples we want for each class
target = min(class_counts.values()) * 10
# Upsample strategies
sampling_strategy_up = {k: target for k, v in class_counts.items() if v < target}
# Downsample strategies
sampling_strategy_down = {k: target for k, v in class_counts.items() if v >
target}
print("Upsample strategies:", sampling_strategy_up)#{'Grayhole': 31570,
'Blackhole': 31570, 'TDMA': 31570, 'Flooding': 31570}
print("Downsample strategies:", sampling_strategy_down)#{'Normal': 31570}
 3 / 25

 # Define the resampling pipeline
resample_pipeline = Pipeline([
    ('smote', SMOTE(sampling_strategy=sampling_strategy_up, random_state=42)),
    ('under', RandomUnderSampler(sampling_strategy=sampling_strategy_down,
random_state=42))
])
# Apply the resampling
X_resampled, y_resampled = resample_pipeline.fit_resample(X, y)

In [ ]:
 # scaling the data
scaler = MinMaxScaler()
X_resampled = scaler.fit_transform(X_resampled)
# save scaller for later use
 with open('later-use/scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [ ]:
  # spliting data
 X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled,
test_size=0.2, random_state=42)

In [ ]:
 def evaluate_model(model,title, X_test, y_test):
    score = model.score(X_test, y_test)
    print(f'{title} Accuracy: {score}')
    y_pred = model.predict(X_test)
    print(f'{title} Classification Report: \n', classification_report(y_test,
y_pred))

In [ ]:
 def residual_block(x, filters, kernel_size=3):
    y = Conv1D(filters, kernel_size, padding='same')(x)
    y = BatchNormalization()(y)
    y = ReLU()(y)
    y = Conv1D(filters, kernel_size, padding='same')(y)
    y = BatchNormalization()(y)
    out = Add()([x, y])
    out = ReLU()(out)
return out
def ResNet1D(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    x = Conv1D(64, 7, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPooling1D(3)(x)
    x = residual_block(x, 64)


    x = residual_block(x, 64)
    x = GlobalAveragePooling1D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
metrics=['accuracy'])
return model

 ResNet1D_model = ResNet1D(input_shape, num_classes)
metrics_history_ResNet1D = MetricsHistory(val_data)
# models summary
print('ResNet1D_model summary:')
ResNet1D_model.summary()
# train the models
ResNet1D_model.fit(X_train_reshaped, y_train_encoded, epochs=10,batch_size=64,
validation_data=val_data, callbacks=[metrics_history_ResNet1D])
evaluate_deep_model(ResNet1D_model, 'ResNet1D', X_test_reshaped, y_test_encoded)